In [1]:
pip install librosa==0.6.3 numpy soundfile==0.9.0 sklearn pyaudio==0.2.11

Note: you may need to restart the kernel to use updated packages.


In [1]:
import soundfile # to read audio file
import numpy as np
import librosa # to extract speech features
import glob
import os
import pickle # to save model after training
from sklearn.model_selection import train_test_split # for splitting training and testing
from sklearn.neural_network import MLPClassifier # multi-layer perceptron model
from sklearn.metrics import accuracy_score # to measure how good we are
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma or contrast:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
        if contrast:
            contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, contrast))
        if tonnetz:
            tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
            result = np.hstack((result, tonnetz))
    return result

In [21]:
# all emotions on RAVDESS dataset
int2emotion = {
    "01": "neutral",
    "02": "calm",
    "03": "happy",
    "04": "sad",
    "05": "angry",
    "06": "fearful",
    "07": "disgust",
    "08": "surprised"
}

# we allow only these emotions ( feel free to tune this on your need )
AVAILABLE_EMOTIONS = {
    "angry",
    "sad",
    "neutral",
    "happy"
}

def load_data(test_size=0.2):
    X, y = [], []
    for file in glob.glob("data/RAVDESS/Actor_*/*.wav"):
            # get the base name of the audio file
            basename = os.path.basename(file)
            # get the emotion label
            emotion = int2emotion[basename.split("-")[2]]
            # we allow only AVAILABLE_EMOTIONS we set
            if emotion not in AVAILABLE_EMOTIONS:
                continue
            # extract speech features
            features = extract_feature(file, mfcc=True, chroma=True, mel=True)
            # add to data
            X.append(features)
            l={'happy':0.0,'sad':1.0,'neutral':3.0,'angry':4.0}
            y.append(l[emotion])
    
         
    # split the data to training and testing and return it
    return train_test_split(np.array(X), y, test_size=test_size, train_size=0.75,random_state=9)

In [22]:
# load RAVDESS dataset, 75% training 25% testing
X_train, X_test, y_train, y_test = load_data(test_size=0.25)

In [23]:
# print some details
# number of samples in training data
print("[+] Number of training samples:", X_train.shape[0])
# number of samples in testing data
print("[+] Number of testing samples:", X_test.shape[0])
# number of features used
# this is a vector of features extracted 
# using extract_features() function
print("[+] Number of features:", X_train.shape[1])

[+] Number of training samples: 503
[+] Number of testing samples: 168
[+] Number of features: 180


In [24]:
# best model, determined by a grid search

        model_params = {
                            'alpha': 0.01,
                            'batch_size': 256,
                            'epsilon': 1e-08, 
                            'hidden_layer_sizes': (300,), 
                            'learning_rate': 'adaptive', 
                            'max_iter': 500, 
                        }

In [25]:
# initialize Multi Layer Perceptron classifier
# with best parameters ( so far )
model = MLPClassifier(**model_params,random_state=22)


In [26]:
# train the model
print("[*] Training the model...")
model.fit(X_train, y_train)

[*] Training the model...


MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500, random_state=22)

In [27]:
# predict 25% of data to measure how good we are
y_pred = model.predict(X_test)

# calculate the accuracy
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 69.05%


In [28]:
print(classification_report(y_test,y_pred))
print('Confusion Matrix : \n',confusion_matrix(y_test,y_pred), '\n')

              precision    recall  f1-score   support

         0.0       0.85      0.58      0.69        50
         1.0       0.56      0.70      0.63        44
         3.0       0.54      0.50      0.52        26
         4.0       0.78      0.90      0.83        48

    accuracy                           0.69       168
   macro avg       0.69      0.67      0.67       168
weighted avg       0.71      0.69      0.69       168

Confusion Matrix : 
 [[29 10  4  7]
 [ 4 31  6  3]
 [ 0 11 13  2]
 [ 1  3  1 43]] 



In [ ]:
# now we save the model
# make result directory if doesn't exist yet
if not os.path.isdir("result"):
    os.mkdir("result")

pickle.dump(model, open("result/mlp_classifier.model", "wb"))

In [ ]:
pickle.dump(model,open('model.pkl' , 'wb'))

In [ ]:
Speech_Emotion_Recognition_model=pickle.load(open('model.pkl','rb'))

In [29]:
from sklearn.svm import SVC 
from sklearn import metrics
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
svm_predictions = svm_model_linear.predict(X_test) 


print(accuracy_score(y_true=y_test,y_pred=svm_predictions))
print(classification_report(y_test,svm_predictions)) 
# creating a confusion matrix 
print(confusion_matrix(y_test, svm_predictions) )

0.6011904761904762
              precision    recall  f1-score   support

         0.0       0.57      0.58      0.57        50
         1.0       0.51      0.55      0.53        44
         3.0       0.54      0.50      0.52        26
         4.0       0.76      0.73      0.74        48

    accuracy                           0.60       168
   macro avg       0.60      0.59      0.59       168
weighted avg       0.60      0.60      0.60       168

[[29 10  2  9]
 [10 24  8  2]
 [ 4  9 13  0]
 [ 8  4  1 35]]


In [30]:
print('Accuracy of model =',accuracy_score(y_test,svm_predictions))

Accuracy of model = 0.6011904761904762


In [31]:
import numpy as np
X_train = np.asarray(X_train)
y_train= np.asarray(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [32]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((503, 180), (503,), (168, 180), (168,))

In [33]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)


In [34]:
x_traincnn.shape,x_testcnn.shape

((503, 180, 1), (168, 180, 1))

In [35]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv1D(128, 5,padding='same',input_shape=(180,1)))        #1
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))



model.add(Conv1D(128, 5,padding='same',))                           #2
model.add(Activation('relu'))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(8))                                                 #3
model.add(Activation('softmax'))
opt = keras.optimizers.RMSprop(lr=0.00005, rho=0.9, epsilon=None, decay=0.0)

In [36]:

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 180, 128)          768       
_________________________________________________________________
activation (Activation)      (None, 180, 128)          0         
_________________________________________________________________
dropout (Dropout)            (None, 180, 128)          0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 22, 128)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 22, 128)           82048     
_________________________________________________________________
activation_1 (Activation)    (None, 22, 128)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 128)           0

In [40]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [41]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=20, epochs=500, validation_data=(x_testcnn, y_test))

Epoch 1/500
26/26 [==============================] - 0s 16ms/step - loss: 2.7836 - accuracy: 0.2525 - val_loss: 1.9014 - val_accuracy: 0.2381
Epoch 2/500
26/26 [==============================] - 0s 11ms/step - loss: 2.4040 - accuracy: 0.3241 - val_loss: 1.4407 - val_accuracy: 0.3036
Epoch 3/500
26/26 [==============================] - 0s 11ms/step - loss: 2.2779 - accuracy: 0.2962 - val_loss: 1.2703 - val_accuracy: 0.3810
Epoch 4/500
26/26 [==============================] - 0s 10ms/step - loss: 2.1610 - accuracy: 0.3459 - val_loss: 1.2738 - val_accuracy: 0.4345
Epoch 5/500
26/26 [==============================] - 0s 10ms/step - loss: 2.0840 - accuracy: 0.3519 - val_loss: 1.6209 - val_accuracy: 0.3333
Epoch 6/500
26/26 [==============================] - 0s 11ms/step - loss: 2.0796 - accuracy: 0.3221 - val_loss: 1.4331 - val_accuracy: 0.3036
Epoch 7/500
26/26 [==============================] - 0s 10ms/step - loss: 1.9967 - accuracy: 0.3777 - val_loss: 1.2547 - val_accuracy: 0.4583
Epoch 

26/26 [==============================] - 0s 11ms/step - loss: 0.9817 - accuracy: 0.5308 - val_loss: 0.9840 - val_accuracy: 0.5000
Epoch 59/500
26/26 [==============================] - 0s 11ms/step - loss: 1.0130 - accuracy: 0.5427 - val_loss: 0.9860 - val_accuracy: 0.5536
Epoch 60/500
26/26 [==============================] - 0s 10ms/step - loss: 1.0312 - accuracy: 0.5229 - val_loss: 0.9848 - val_accuracy: 0.5476
Epoch 61/500
26/26 [==============================] - 0s 11ms/step - loss: 0.9933 - accuracy: 0.5268 - val_loss: 1.0670 - val_accuracy: 0.5476
Epoch 62/500
26/26 [==============================] - 0s 11ms/step - loss: 0.9989 - accuracy: 0.5567 - val_loss: 0.9540 - val_accuracy: 0.6012
Epoch 63/500
26/26 [==============================] - 0s 11ms/step - loss: 0.9961 - accuracy: 0.5547 - val_loss: 0.9637 - val_accuracy: 0.5000
Epoch 64/500
26/26 [==============================] - 0s 10ms/step - loss: 0.9647 - accuracy: 0.5686 - val_loss: 1.0420 - val_accuracy: 0.5060
Epoch 65/500

26/26 [==============================] - 0s 10ms/step - loss: 0.8109 - accuracy: 0.6362 - val_loss: 0.8953 - val_accuracy: 0.6012
Epoch 116/500
26/26 [==============================] - 0s 10ms/step - loss: 0.8328 - accuracy: 0.6163 - val_loss: 0.8734 - val_accuracy: 0.5833
Epoch 117/500
26/26 [==============================] - 0s 11ms/step - loss: 0.8278 - accuracy: 0.6203 - val_loss: 0.8786 - val_accuracy: 0.5893
Epoch 118/500
26/26 [==============================] - 0s 10ms/step - loss: 0.8253 - accuracy: 0.6143 - val_loss: 0.8842 - val_accuracy: 0.6310
Epoch 119/500
26/26 [==============================] - 0s 11ms/step - loss: 0.8170 - accuracy: 0.6421 - val_loss: 0.8841 - val_accuracy: 0.6012
Epoch 120/500
26/26 [==============================] - 0s 11ms/step - loss: 0.8283 - accuracy: 0.6541 - val_loss: 0.8671 - val_accuracy: 0.6190
Epoch 121/500
26/26 [==============================] - 0s 11ms/step - loss: 0.8165 - accuracy: 0.6382 - val_loss: 0.9267 - val_accuracy: 0.5833
Epoch 

Epoch 172/500
26/26 [==============================] - 0s 11ms/step - loss: 0.7397 - accuracy: 0.7018 - val_loss: 0.8572 - val_accuracy: 0.6667
Epoch 173/500
26/26 [==============================] - 0s 10ms/step - loss: 0.7420 - accuracy: 0.6799 - val_loss: 0.8287 - val_accuracy: 0.6369
Epoch 174/500
26/26 [==============================] - 0s 10ms/step - loss: 0.7519 - accuracy: 0.6740 - val_loss: 0.8538 - val_accuracy: 0.6310
Epoch 175/500
26/26 [==============================] - 0s 11ms/step - loss: 0.7001 - accuracy: 0.7038 - val_loss: 0.9195 - val_accuracy: 0.5774
Epoch 176/500
26/26 [==============================] - 0s 11ms/step - loss: 0.7181 - accuracy: 0.7018 - val_loss: 0.8975 - val_accuracy: 0.6131
Epoch 177/500
26/26 [==============================] - 0s 11ms/step - loss: 0.7159 - accuracy: 0.6918 - val_loss: 0.8611 - val_accuracy: 0.5893
Epoch 178/500
26/26 [==============================] - 0s 11ms/step - loss: 0.7218 - accuracy: 0.6899 - val_loss: 0.9055 - val_accuracy:

Epoch 229/500
26/26 [==============================] - 0s 11ms/step - loss: 0.6412 - accuracy: 0.7416 - val_loss: 0.8427 - val_accuracy: 0.6071
Epoch 230/500
26/26 [==============================] - 0s 11ms/step - loss: 0.6059 - accuracy: 0.7634 - val_loss: 0.8589 - val_accuracy: 0.6012
Epoch 231/500
26/26 [==============================] - 0s 11ms/step - loss: 0.6428 - accuracy: 0.7376 - val_loss: 0.8237 - val_accuracy: 0.6548
Epoch 232/500
26/26 [==============================] - 0s 11ms/step - loss: 0.6316 - accuracy: 0.7555 - val_loss: 0.8112 - val_accuracy: 0.6488
Epoch 233/500
26/26 [==============================] - 0s 12ms/step - loss: 0.6559 - accuracy: 0.7237 - val_loss: 0.8380 - val_accuracy: 0.6250
Epoch 234/500
26/26 [==============================] - 0s 11ms/step - loss: 0.6026 - accuracy: 0.7813 - val_loss: 0.8234 - val_accuracy: 0.6310
Epoch 235/500
26/26 [==============================] - 0s 12ms/step - loss: 0.6227 - accuracy: 0.7495 - val_loss: 0.8541 - val_accuracy:

Epoch 286/500
26/26 [==============================] - 0s 12ms/step - loss: 0.5628 - accuracy: 0.7833 - val_loss: 0.8475 - val_accuracy: 0.6250
Epoch 287/500
26/26 [==============================] - 0s 12ms/step - loss: 0.5640 - accuracy: 0.7694 - val_loss: 0.8680 - val_accuracy: 0.6071
Epoch 288/500
26/26 [==============================] - 0s 12ms/step - loss: 0.5447 - accuracy: 0.8032 - val_loss: 0.8288 - val_accuracy: 0.6429
Epoch 289/500
26/26 [==============================] - 0s 12ms/step - loss: 0.5737 - accuracy: 0.7555 - val_loss: 0.8379 - val_accuracy: 0.6310
Epoch 290/500
26/26 [==============================] - 0s 12ms/step - loss: 0.5799 - accuracy: 0.7734 - val_loss: 0.8759 - val_accuracy: 0.6012
Epoch 291/500
26/26 [==============================] - 0s 12ms/step - loss: 0.5461 - accuracy: 0.7773 - val_loss: 0.8187 - val_accuracy: 0.6607
Epoch 292/500
26/26 [==============================] - 0s 12ms/step - loss: 0.5644 - accuracy: 0.7833 - val_loss: 0.8600 - val_accuracy:

Epoch 343/500
26/26 [==============================] - 0s 12ms/step - loss: 0.4797 - accuracy: 0.8091 - val_loss: 0.8079 - val_accuracy: 0.6548
Epoch 344/500
26/26 [==============================] - 0s 13ms/step - loss: 0.5135 - accuracy: 0.8012 - val_loss: 0.8157 - val_accuracy: 0.6369
Epoch 345/500
26/26 [==============================] - 0s 13ms/step - loss: 0.4866 - accuracy: 0.8290 - val_loss: 0.8130 - val_accuracy: 0.6488
Epoch 346/500
26/26 [==============================] - 0s 12ms/step - loss: 0.4850 - accuracy: 0.8310 - val_loss: 0.8221 - val_accuracy: 0.6667
Epoch 347/500
26/26 [==============================] - 0s 12ms/step - loss: 0.4750 - accuracy: 0.8390 - val_loss: 0.8692 - val_accuracy: 0.6310
Epoch 348/500
26/26 [==============================] - 0s 12ms/step - loss: 0.5051 - accuracy: 0.7932 - val_loss: 0.8028 - val_accuracy: 0.6667
Epoch 349/500
26/26 [==============================] - 0s 12ms/step - loss: 0.5047 - accuracy: 0.8091 - val_loss: 0.8130 - val_accuracy:

Epoch 400/500
26/26 [==============================] - 0s 12ms/step - loss: 0.4193 - accuracy: 0.8688 - val_loss: 0.8372 - val_accuracy: 0.6607
Epoch 401/500
26/26 [==============================] - 0s 12ms/step - loss: 0.4631 - accuracy: 0.8350 - val_loss: 0.7787 - val_accuracy: 0.6667
Epoch 402/500
26/26 [==============================] - 0s 12ms/step - loss: 0.4210 - accuracy: 0.8469 - val_loss: 0.7893 - val_accuracy: 0.6667
Epoch 403/500
26/26 [==============================] - 0s 12ms/step - loss: 0.4348 - accuracy: 0.8429 - val_loss: 0.8156 - val_accuracy: 0.6667
Epoch 404/500
26/26 [==============================] - 0s 12ms/step - loss: 0.4173 - accuracy: 0.8608 - val_loss: 0.7963 - val_accuracy: 0.6786
Epoch 405/500
26/26 [==============================] - 0s 12ms/step - loss: 0.4520 - accuracy: 0.8469 - val_loss: 0.7895 - val_accuracy: 0.6667
Epoch 406/500
26/26 [==============================] - 0s 12ms/step - loss: 0.4190 - accuracy: 0.8429 - val_loss: 0.7963 - val_accuracy:

Epoch 457/500
26/26 [==============================] - 0s 12ms/step - loss: 0.3711 - accuracy: 0.8787 - val_loss: 0.8027 - val_accuracy: 0.6905
Epoch 458/500
26/26 [==============================] - 0s 12ms/step - loss: 0.3739 - accuracy: 0.8588 - val_loss: 0.7975 - val_accuracy: 0.6667
Epoch 459/500
26/26 [==============================] - 0s 13ms/step - loss: 0.3811 - accuracy: 0.8688 - val_loss: 0.7917 - val_accuracy: 0.6786
Epoch 460/500
26/26 [==============================] - 0s 12ms/step - loss: 0.3636 - accuracy: 0.8807 - val_loss: 0.8290 - val_accuracy: 0.6310
Epoch 461/500
26/26 [==============================] - 0s 12ms/step - loss: 0.3949 - accuracy: 0.8529 - val_loss: 0.8034 - val_accuracy: 0.6845
Epoch 462/500
26/26 [==============================] - 0s 12ms/step - loss: 0.3644 - accuracy: 0.8807 - val_loss: 0.8563 - val_accuracy: 0.6488
Epoch 463/500
26/26 [==============================] - 0s 12ms/step - loss: 0.3603 - accuracy: 0.8827 - val_loss: 0.8330 - val_accuracy:

In [53]:
em=['happy','sad','neutral','angry']

In [56]:
predictions = model.predict_classes(x_testcnn)
n=predictions[1]
print(em[n])
print(n)

sad
1


In [57]:
loss, acc = model.evaluate(x_testcnn, y_test)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

6/6 [==============================] - 0s 3ms/step - loss: 0.8024 - accuracy: 0.6905
Restored model, accuracy: 69.05%


In [45]:
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
predictions = model.predict_classes(x_testcnn)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))

              precision    recall  f1-score   support

         0.0       0.80      0.66      0.73        50
         1.0       0.59      0.80      0.68        44
         3.0       0.56      0.38      0.45        26
         4.0       0.76      0.79      0.78        48

    accuracy                           0.69       168
   macro avg       0.68      0.66      0.66       168
weighted avg       0.70      0.69      0.69       168

[[33  6  2  9]
 [ 2 35  5  2]
 [ 1 14 10  1]
 [ 5  4  1 38]]


In [58]:
filename = "data/RAVDESS/Actor_02/03-01-01-01-02-01-02.wav"
    # record the file (start talking)
    #record_to_file(filename)
    # extract features and reshape it
features = np.array(extract_feature(filename, mfcc=True, chroma=True, mel=True).reshape(1, -1))
    # predict
f=np.expand_dims(features,axis=2)
result = model.predict_classes(f)[0]
    # show the result !
print("result :",em[result])

IndexError: list index out of range